Lien de téléchargement des données: https://cvml.ista.ac.at/AwA2/        

13GB file : https://cvml.ista.ac.at/AwA2/AwA2-data.zip

In [1]:
import sys
import cv2 # Pour utiliser open_cv, il faut la version de python est 3.7
import os
import csv

import numpy as np 
import pandas as pd 
import math

import torch 
from torch.utils.data import Dataset, DataLoader
import torchvision 
from torchvision.io import read_image
import torchvision.datasets as datasets
import torchvision.transforms as transforms

np.random.seed(0)

In [2]:
# Constant. Should be the path to the folder named JPEGImages, containing the 33K images in its subfolders.
DATA_FOLDER_PATH = "/Users/xiongyuyang/Downloads/Animals_with_Attributes2/"
JPEGIMAGES_FOLDER_PATH = "/Users/xiongyuyang/Downloads/Animals_with_Attributes2/JPEGImages/"

In [3]:
# quick test
test = JPEGIMAGES_FOLDER_PATH+"fox/fox_10001.jpg"
img = cv2.imread(test) 
print(img.shape) #ndarray
print(type(img))
cv2.imshow('Sample Image from AwA2 dataset',img)
cv2.waitKey(0)

(764, 918, 3)
<class 'numpy.ndarray'>


In [3]:
labels_dirs = os.listdir(JPEGIMAGES_FOLDER_PATH)
labels_dirs = [f for f in labels_dirs if not f.startswith('.')] # remove hidden files
print(labels_dirs)
len(labels_dirs) # 50 labels / subdirectories

['hippopotamus', 'gorilla', 'rhinoceros', 'raccoon', 'humpback+whale', 'killer+whale', 'pig', 'giant+panda', 'dalmatian', 'moose', 'antelope', 'persian+cat', 'siamese+cat', 'fox', 'dolphin', 'polar+bear', 'giraffe', 'zebra', 'leopard', 'blue+whale', 'chihuahua', 'sheep', 'hamster', 'ox', 'tiger', 'rat', 'otter', 'spider+monkey', 'collie', 'seal', 'mouse', 'beaver', 'wolf', 'chimpanzee', 'grizzly+bear', 'buffalo', 'bobcat', 'rabbit', 'bat', 'weasel', 'walrus', 'mole', 'horse', 'skunk', 'german+shepherd', 'squirrel', 'cow', 'deer', 'lion', 'elephant']


50

# Note : Some labels have a low number of images. 

## Possible solutions to explore : 
    Data augmentation : creating new training data by applying random transformations to existing images, such as rotating, cropping, or flipping them.

In [4]:
def find_num_images_per_label(img_dir = JPEGIMAGES_FOLDER_PATH) -> tuple[dict,dict]: 
    """ 
    USEFUL FOR SAMPLING.
    Return a dict with keys as the 50 labels, and values being the number of images in each subdirectory corresponding to label
    and a second dict with the relative numbers (proportion) for every label compared to the total number of images (useful for sampling)"""
    labels_dirs = os.listdir(img_dir)
    labels_dirs = [f for f in labels_dirs if not f.startswith('.')] # remove hidden files
    num_images_per_label = dict.fromkeys(labels_dirs)
    proportions_images_per_label = dict.fromkeys(labels_dirs)
    total_num_images = 0

    # Update absolute number of images per label
    for i, label in enumerate(labels_dirs) : 
        specific_label_path = os.path.join(img_dir, labels_dirs[i])
        num_images_label = len(os.listdir(specific_label_path))
        total_num_images += num_images_label
        num_images_per_label[label] = num_images_label

    # Update relative number of images per label (proportion)
    for i, label in enumerate(labels_dirs) : 
        num_images_label = num_images_per_label[label]
        proportion_label = round(num_images_label / total_num_images, 4)
        proportions_images_per_label[label] = proportion_label

    return num_images_per_label, proportions_images_per_label

num_images_per_label, proportions_images_per_label = find_num_images_per_label()
print(num_images_per_label)
print(proportions_images_per_label)

{'hippopotamus': 684, 'gorilla': 872, 'rhinoceros': 696, 'raccoon': 512, 'humpback+whale': 709, 'killer+whale': 291, 'pig': 713, 'giant+panda': 874, 'dalmatian': 549, 'moose': 704, 'antelope': 1046, 'persian+cat': 747, 'siamese+cat': 500, 'fox': 664, 'dolphin': 946, 'polar+bear': 868, 'giraffe': 1202, 'zebra': 1170, 'leopard': 720, 'blue+whale': 174, 'chihuahua': 567, 'sheep': 1420, 'hamster': 779, 'ox': 728, 'tiger': 877, 'rat': 310, 'otter': 758, 'spider+monkey': 291, 'collie': 1028, 'seal': 988, 'mouse': 185, 'beaver': 193, 'wolf': 589, 'chimpanzee': 728, 'grizzly+bear': 852, 'buffalo': 895, 'bobcat': 630, 'rabbit': 1088, 'bat': 383, 'weasel': 272, 'walrus': 215, 'mole': 100, 'horse': 1645, 'skunk': 188, 'german+shepherd': 1033, 'squirrel': 1200, 'cow': 1338, 'deer': 1344, 'lion': 1019, 'elephant': 1038}
{'hippopotamus': 0.0183, 'gorilla': 0.0234, 'rhinoceros': 0.0186, 'raccoon': 0.0137, 'humpback+whale': 0.019, 'killer+whale': 0.0078, 'pig': 0.0191, 'giant+panda': 0.0234, 'dalmatia

In [5]:
ANNOTATIONS_FILENAME = 'annotations.csv'

def create_annotations_csv_file(annotations_filename = ANNOTATIONS_FILENAME, img_dir = JPEGIMAGES_FOLDER_PATH): 
    """ 
    Create a csv annotations_file, annotations.csv, with two columns, in the format : 
                        path/to/image, label
    
    The annotation csv is necessary for DataLoader.
    """
    
    labels_dirs:list = os.listdir(img_dir)
    labels_dirs = [f for f in labels_dirs if not f.startswith('.')] # remove hidden files
   
    if os.path.exists(annotations_filename):
        os.remove(annotations_filename)
        print(f'Deleted existent {ANNOTATIONS_FILENAME} file.\n ---------------------------')
    
    with open(annotations_filename, 'w', newline='') as file :
        writer = csv.writer(file, dialect='excel', delimiter=',')

        for i, label in enumerate(labels_dirs) : 

            specific_label_path = os.path.join(img_dir, label)
            images_names = os.listdir(specific_label_path)

            for j, image_name in enumerate(images_names):
                full_path_to_img= os.path.join(specific_label_path, image_name)
                full_path_to_img= os.path.join(label, image_name)

                row = [full_path_to_img, label]
                writer.writerow(row)

    print(f'Sucessfully created {ANNOTATIONS_FILENAME} file.')

#
create_annotations_csv_file()

Deleted existent annotations.csv file.
 ---------------------------
Sucessfully created annotations.csv file.


In [6]:
# labels_in_number = pd.read_csv(DATA_FOLDER_PATH+"classes.txt", delim_whitespace=True,header=None)
labels_dict = {}
with open(DATA_FOLDER_PATH+"classes.txt") as f:
    for line in f:
        # print(line.split())
        (key,val) = line.split()
        labels_dict[val] = int(key)-1
print(labels_dict)

{'antelope': 0, 'grizzly+bear': 1, 'killer+whale': 2, 'beaver': 3, 'dalmatian': 4, 'persian+cat': 5, 'horse': 6, 'german+shepherd': 7, 'blue+whale': 8, 'siamese+cat': 9, 'skunk': 10, 'mole': 11, 'tiger': 12, 'hippopotamus': 13, 'leopard': 14, 'moose': 15, 'spider+monkey': 16, 'humpback+whale': 17, 'elephant': 18, 'gorilla': 19, 'ox': 20, 'fox': 21, 'sheep': 22, 'seal': 23, 'chimpanzee': 24, 'hamster': 25, 'squirrel': 26, 'rhinoceros': 27, 'rabbit': 28, 'bat': 29, 'giraffe': 30, 'wolf': 31, 'chihuahua': 32, 'rat': 33, 'weasel': 34, 'otter': 35, 'buffalo': 36, 'zebra': 37, 'giant+panda': 38, 'deer': 39, 'bobcat': 40, 'pig': 41, 'lion': 42, 'mouse': 43, 'polar+bear': 44, 'collie': 45, 'walrus': 46, 'raccoon': 47, 'cow': 48, 'dolphin': 49}


In [7]:
class AWA2Dataset(Dataset): # Dataset class to serve as input for the DataLoader.
    """ 
    Dataset class to serve as input for the DataLoader.
    Implements all the required methods and more. 
    """

    def __init__(self, annotations_file=ANNOTATIONS_FILENAME, img_dir=JPEGIMAGES_FOLDER_PATH, 
                transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

        numbers_infos_dicts: tuple[dict,dict] = find_num_images_per_label(img_dir=JPEGIMAGES_FOLDER_PATH)
        self.num_images_per_label = numbers_infos_dicts[0]
        self.proportions_images_per_label = numbers_infos_dicts[1]

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        # img_path = self.img_labels.iloc[idx, 0]
        key = self.img_labels.iloc[idx, 1]

        # Mapping the labels from string to tensor
        label = labels_dict[key]

        image = read_image(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [8]:
dataset = AWA2Dataset()
image,label = dataset[4125]
print(label) #
print(image.shape)#
print(image)#
print(len(dataset))#

## TODO : Change transforms. Currently this is not useful.
dataset.transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize(256),
                    transforms.CenterCrop(224),
                    transforms.Grayscale(num_output_channels=3),
                    transforms.ToTensor(), # Already a tensor as implemented in Dataset class with the reaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
                    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                ])

# Testing. All good
# random_index = np.random.randint(0, len(dataset))
# image, label = dataset[1]
# print(label)
# print(image)
train_size =  int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size,test_size])

41
torch.Size([3, 574, 400])
tensor([[[114, 117, 118,  ..., 251, 241, 245],
         [120, 113, 134,  ..., 248, 248, 246],
         [115, 114, 123,  ..., 247, 249, 250],
         ...,
         [118, 108, 109,  ..., 131, 133, 122],
         [122, 118, 126,  ..., 139, 123, 126],
         [118, 125, 122,  ..., 116, 134, 133]],

        [[137, 137, 136,  ..., 245, 247, 239],
         [135, 138, 139,  ..., 249, 240, 247],
         [138, 135, 150,  ..., 249, 247, 241],
         ...,
         [139, 135, 138,  ..., 136, 140, 141],
         [137, 141, 141,  ..., 150, 138, 137],
         [138, 136, 138,  ..., 136, 139, 139]],

        [[ 57,  48,  74,  ..., 245, 237, 241],
         [ 66,  54,  45,  ..., 235, 253, 233],
         [ 50,  56,  57,  ..., 248, 234, 246],
         ...,
         [ 83,  84,  74,  ...,  78,  70,  85],
         [104,  85,  76,  ...,  84,  83,  68],
         [ 85, 102,  76,  ...,  73,  73,  77]]], dtype=torch.uint8)
37321


In [9]:
# Experiment with DataLoader. Everything works good
dataloader = DataLoader(dataset = dataset, batch_size=4, shuffle=True)
dataiter = iter(dataloader)
data = next(dataiter)

images, labels = data 
#print(dataloader)#
print(labels, images.shape)
#print(labels)

tensor([17, 48, 22,  5]) torch.Size([4, 3, 224, 224])


In [10]:
# Training loop example
num_epochs = 2 
batch_size = 4
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/batch_size)
print(total_samples, n_iterations)

dataloader = DataLoader(dataset = dataset, batch_size=batch_size, shuffle=True)

for epoch in range(num_epochs) : 
    # loop over trainloader 
    for i, (inputs, labels) in enumerate(dataloader) : 
        
        # Do forward and backward pass, update the weights 
        if(i+1) % 5 == 0 :
            print(f'epoch {epoch+1} / {num_epochs}, step, {i+1}/{n_iterations}, inputs {inputs.shape}')

        if i==20 : 
            print('Completed')
            break


37321 9331
epoch 1 / 2, step, 5/9331, inputs torch.Size([4, 3, 224, 224])
epoch 1 / 2, step, 10/9331, inputs torch.Size([4, 3, 224, 224])
epoch 1 / 2, step, 15/9331, inputs torch.Size([4, 3, 224, 224])
epoch 1 / 2, step, 20/9331, inputs torch.Size([4, 3, 224, 224])
Completed
epoch 2 / 2, step, 5/9331, inputs torch.Size([4, 3, 224, 224])
epoch 2 / 2, step, 10/9331, inputs torch.Size([4, 3, 224, 224])
epoch 2 / 2, step, 15/9331, inputs torch.Size([4, 3, 224, 224])
epoch 2 / 2, step, 20/9331, inputs torch.Size([4, 3, 224, 224])
Completed


In [12]:
from torchvision import models
resnet = models.resnet18(pretrained=True)
print(resnet)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/xiongyuyang/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=50):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle= True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle= True)
print(len(train_dataset)) 
print(len(test_dataset))

print(len(train_dataloader), len(test_dataloader))
model = ResNet18()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 2
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_dataloader):
        print(inputs.shape, labels.shape)
        optimizer.zero_grad()
        outputs = resnet(inputs)
        print(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print training statistics
        if (i+1) % 500== 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}")



29856
7465
7464 1867
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size([4])
1
torch.Size([4, 3, 224, 224]) torch.Size

In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=16)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=32)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=64)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2)

        # Fully connected layers
        self.fc1 = nn.Linear(in_features=64 * 28 * 28, out_features=512)
        self.relu4 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(in_features=512, out_features=50)

    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)

        # Fully connected layers
        x = torch.flatten(x, start_dim=1)
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle= True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle= True)

model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 2
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_dataloader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print training statistics
        if (i+1) % 500== 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_dataloader)}], Loss: {loss.item():.4f}")


KeyboardInterrupt: 

In [15]:

resnet.eval()  # switch to evaluation mode

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:

        outputs = resnet(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        print('Accuracy of the model on the test images: {:.2f}%'.format(100 * correct / total))

Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images: 0.00%
Accuracy of the model on the test images